In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = utils.reduce_mem_usage(df_hist)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = utils.reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)

100%|██████████| 3/3 [00:23<00:00,  7.82s/it]


Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(29112361, 21)
(29112361, 22)


In [9]:
# term
term_num = 13
base_list = np.zeros(term_num)
base_list = np.arange(1, term_num, 1)
last_list = np.arange(3, term_num, 1)
col_month = 'org_month_lag'

from dateutil.parser import parse
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for base_month, last_month in zip(tqdm(base_list), last_list):
    prefix = f'175_l{base_month}{last_month}'
    print(f"prefix: {prefix}")

    df = df_hist[df_hist[col_month]<=last_month]
    df = df[df[col_month]>=base_month]
    
    auth1 = df[df.authorized_flag==1]
    auth0 = df[df.authorized_flag==0]
    
    cat1_0 = False
    # cat1_0 = True
    if cat1_0:
        auth1_cat1 = auth1[auth1.category_1==1]
        auth1_cat0 = auth1[auth1.category_1==0]
        auth0_cat1 = auth0[auth0.category_1==1]
        auth0_cat0 = auth0[auth0.category_1==0]
        df_list = [ auth1_cat1 ,auth1_cat0 ,auth0_cat1 ,auth0_cat0 ]
        fname_list = ['auth1_cat1', 'auth1_cat0', 'auth0_cat1', 'auth0_cat0',]
    else:
        df_list = [auth1, auth0]
        fname_list = ['auth1', 'auth0']

    for df, base_fname in zip(tqdm(df_list), fname_list):
        fname = f'org_lag{base_month}_{last_month}_' + base_fname
        print(f"fname: {fname}")
        
        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        train_test = pd.concat([df_train, df_test], axis=0)
        train_test.set_index(key, inplace=True)
        
#         df['purchase_date'] = pd.to_datetime(df['purchase_date'])
#         df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)
        
#         df.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
        
#         #========================================================================
#         # Feature Engineering
#         #========================================================================
#         aggs = {}
#         aggs['date_diff'] = ['mean', 'std']
        
#         df = df[[key, 'purchase_date', 'date_diff', 'yyyymm']]
        
#         ### 日付は関係なく、直近何回、初回から何回の購買におけるデータを集計する
#         # first 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する  
#         # last 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する  
#         # 同日購買も重複削除せずに集計するversion
    
#         for i in range(5, 26, 5):
            
#             df.sort_values(by=[key, 'purchase_date'], inplace=True)
#             df = utils.row_number(df=df, level=key)
        
#             feat_n = f'{fname}_first'
#             first = df.query(f"row_no<={i}")
#             tmp = first.groupby(key)['date_diff'].agg(aggs)
#             new_cols = get_new_columns(f'{feat_n}{i}', aggs)
#             tmp.columns = new_cols
#             train_test = train_test.join(tmp)
            
            
#         for i in range(5, 26, 5):
            
#             df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
#             df = utils.row_number(df=df, level=key)
            
#             feat_n = f'{fname}_last'
#             last = df.query(f"row_no<={i}")
#             tmp = last.groupby(key)['date_diff'].agg(aggs)
#             new_cols = get_new_columns(f'{feat_n}{i}', aggs)
#             tmp.columns = new_cols
#             train_test = train_test.join(tmp)
            
            
        #========================================================================
        # 132 date_diffの前半、後半における違いを特徴にする. 全体verも
        #========================================================================
            
        # 日付の重複を削除するため
        df['purchase_date'] =  df['purchase_date'].map(lambda x: str(x)[:10])
        
        # 前回購買日との差分
        def diff_shift_date(df, lag):
            
            # shiftは正が下方向、負が上方向にズレる
            df = df[[key, 'purchase_date']].drop_duplicates()
            df.sort_values(by=[key, 'purchase_date'], inplace=True)
            df['purchase_date'] = pd.to_datetime(df['purchase_date'])
            df['shift1_date'] = df.groupby(key)['purchase_date'].shift(lag)
            df = df[[key, 'purchase_date', 'shift1_date']].dropna()
            df['date_diff_lag1'] = (df['purchase_date'] - df['shift1_date']).dt.days
            return df
            
        diff = diff_shift_date(df, 1)
            
        df = diff[[key, 'purchase_date', 'date_diff_lag1']].drop_duplicates()
    
        aggs = {}
        aggs['date_diff_lag1'] = ['mean', 'std']
    
        df.sort_values(by=[key, 'purchase_date'], inplace=True)
        df = utils.row_number(df=df, level=key)
        
        for i in range(5, 20, 5):
            
            feat_n = f'{fname}_first'
            first = df.query(f"row_no<={i}")
            tmp = first.groupby(key)['date_diff_lag1'].agg(aggs)
            new_cols = get_new_columns(f'{feat_n}{i}', aggs)
            tmp.columns = new_cols
            train_test = train_test.join(tmp)
        
        df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
        df = utils.row_number(df=df, level=key)
        for i in range(5, 20, 5):
             
            feat_n = f'{fname}_last'
            last = df.query(f"row_no<={i}")
            tmp = last.groupby(key)['date_diff_lag1'].agg(aggs)
            new_cols = get_new_columns(f'{feat_n}{i}', aggs)
            tmp.columns = new_cols
            train_test = train_test.join(tmp)
            
        #========================================================================
        # Saveing
        #========================================================================
        ignore_features = ['first_active_month', 'card_id']
        
        train_test.fillna(-1, inplace=True)
        
        for col in train_test.columns:
            if col in ignore_features: continue
            if not(col.count('date_diff')):continue
        
            feature = train_test[col].values.astype('float32')
            utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)
        #     utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)




  0%|          | 0/12 [00:00<?, ?it/s]

prefix: 175_l13






  0%|          | 0/2 [00:00<?, ?it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 116.16it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 165.95it/s]

fname: org_lag1_3_auth1






 50%|█████     | 1/2 [00:49<00:49, 49.46s/it]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 108.40it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 175.40it/s]

...





  0%|          | 0/2 [00:00<?, ?it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 107.27it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 167.51it/s]

fname: org_lag10_12_auth1






 50%|█████     | 1/2 [00:21<00:21, 21.55s/it]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 103.40it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




100%|██████████| 3/3 [00:00<00:00, 160.46it/s]

fname: org_lag10_12_auth0






100%|██████████| 2/2 [00:26<00:00, 16.64s/it]






 83%|████████▎ | 10/12 [07:33<01:15, 37.54s/it]